# Welcome to Google Colab Re-Vanced Builder!
## About me
Hi! I'm **[Aswin](https://github.com/Jarvis-Ank)**.
I created this project to make it easier for people with less skills to build their own version of ReVanced from already prepared libraries.
If you like my work, visit my profile and project page and leave a ⭐ and a follow.
* My GitHub: https://github.com/Jarvis-Ank
* Project URL: https://github.com/Jarvis-Ank/Re-Vanced

---

## What is YOUTUBE_ID?
**YOUTUBE_ID** is the identifier of the file you get after uploading the file to **Google Drive** and sharing it. It is required to create your version of the ReVanced client.
## How to get YOUTUBE_ID?
To get the ID you need, please follow the steps below:
1. Download the Youtube apk file from [ApkMirror](https://www.apkmirror.com/uploads/?appcategory=youtube)
2. Upload the downloaded file to your [Google Drive](https://drive.google.com)
3. Rename the file to "***youtube.apk***"
4. Share the file via the "***Get link***" option
5. Change the file accessibility to "***Anyone with the link***"
6. Now you just need to copy the generated link and paste it as YOUTUBE_ID



In [ ]:
#@title **Set variables** { vertical-output: true, display-mode: "form" }
YOUTUBE_ID = "" #@param {type: "string"}
INTEGRATIONS_TAG = "latest" #@param {type: "string"}
PATCHES_TAG = "latest" #@param {type: "string"}
CLI_TAG = "latest" #@param {type: "string"}
MICROG_URL = "default" #@param {type: "string"}

from subprocess import run
import requests
def exec(cmd,show_output=False):
  if show_output:
    run(cmd, shell=True, check=True)
  else:
    out = run(cmd, shell=True, capture_output=True, text=True, check=True)

def get_tag(repo):
  return requests.get(f'https://api.github.com/repos/revanced/{repo}/releases/latest').json()['name'][1:]

def check_tag(repo, tag):
  tags = []
  for v_tag in requests.get(f'https://api.github.com/repos/revanced/{repo}/tags').json(): tags.append(v_tag['name'])
  if tag in tags: return True
  else:
    print(f'Tag {tag} does\'t exist in the revanced/{repo} repository.\nUsing latest version.')
    return False

def check_yt(url):
  if 'https://drive.google.com/file/d/' not in url:
    url = f'https://drive.google.com/file/d/{url}/view?usp=sharing'
  res = requests.get(url)
  if res.status_code == 200: return True
  else: return False

# Updating system
print('Updating system before continuing')
exec('apt update')
exec('apt upgrade')
print('Update finished')

# Removing old revanced.apk
exec('rm -f ./revanced.apk')

# Creating working directory
print('Creating Directories....')
exec('mkdir ./R_V')
%cd R_V
exec('sleep 1')

# Installing requirements
print('Installing gdown & Zulu JDK-17')
#Drive downloader
exec('pip install gdown')
exec('sleep 3')
#Zulu JDK-17 download and install
exec('wget -q --show-progress -O zulu-jdk17.deb https://cdn.azul.com/zulu/bin/zulu17.34.19-ca-jdk17.0.3-linux_amd64.deb')
exec('sleep 3')
exec('sudo apt install ./zulu-jdk17.deb')
print('Finished installing')
exec('sleep 5')

# Downloading Required files
print('Downloading necessary files')
#Integrations
if INTEGRATIONS_TAG is 'latest' or not check_tag('revanced-integrations', INTEGRATIONS_TAG): INTEGRATIONS_TAG = get_tag('revanced-integrations')
exec(f'wget -q --show-progress -O revanced-integrations.apk https://github.com/revanced/revanced-integrations/releases/download/v{INTEGRATIONS_TAG}/app-release-unsigned.apk', True)
#Patches
if PATCHES_TAG is 'latest' or not check_tag('revanced-patches', PATCHES_TAG): PATCHES_TAG = get_tag('revanced-patches')
exec(f'wget -q --show-progress -O revanced-patches.jar https://github.com/revanced/revanced-patches/releases/download/v{PATCHES_TAG}/revanced-patches-{PATCHES_TAG}.jar', True)
#CLI
if CLI_TAG is 'latest' or not check_tag('revanced-cli', CLI_TAG): CLI_TAG = get_tag('revanced-cli')
exec(f'wget -q --show-progress -O revanced-cli.jar https://github.com/revanced/revanced-cli/releases/download/v{CLI_TAG}/revanced-cli-{CLI_TAG}-all.jar', True)
#Micro-G
if MICROG_URL is 'default': MICROG_URL = 'https://github.com/Jarvis-Ank/Re-Vanced/blob/main/microG_0.2.24.220220.apk'
exec(f'wget -q --show-progress -O microg.apk {MICROG_URL}', True)
print('Download finished')
#Youtube
print('Downloading youtube apk')
retries = 0
while retries < 5:
  try:
    if not check_yt(YOUTUBE_ID): raise Exception('Incorrect Drive ID')
  except:
    print('Google Drive ID is missing or incorrect.\nPlease provide correct id or url: ', end='')
    YOUTUBE_ID = input()
    retries += 1
    continue
  else:
    exec(f'gdown {YOUTUBE_ID}')
    break
print('Download finished')
exec('sleep 5')


# Building revanced.apk
print('Building revanced apk')
exec('java -jar revanced-cli.jar -a youtube.apk -c -o revanced.apk -b revanced-patches.jar -m revanced-integrations.apk')
print('Ended sucessfully')
exec('sleep 3')

print('Moving output file')
%mv revanced.apk ..
print('Moved sucessfully')
exec('sleep 3')

# Cleaning after build
%cd ..
print('Removing unnecessary files')
%rm -rf R_V
print('Removed sucessfully')
exec('sleep 3')

# Finishing script
%pwd
print('Finished sucessfully, check if revanced.apk is in the folder')